In [1]:
from googletrans import Translator

translator = Translator()
translator.translate('Good Morning', dest = 'ko', src = 'en').text

'좋은 아침이에요'

In [ ]:
import deepl
deepl.translate(source_language="EN", target_language="KO", text="Hello World")

In [ ]:
from model import Translator_GoogleGemini

translator = Translator_GoogleGemini()


translator.translate("You see, back in 2009, we all had these weird little things called attention spans.")